## 7.1 - *Adult* [dataset](https://archive.ics.uci.edu/ml/datasets/Adult)

### Carga y Preprocesamiento de los datos


In [ ]:
# Librerías
import os # Para obtener el directorio activo
import requests # Para descargar ficheros
import re
import pandas as pd
import numpy as np

In [ ]:
# Creamos una carpeta para que contenga a nuestro dataset
!mkdir adult_dataset
# Movemos el directorio activo a esa localización
os.chdir("adult_dataset")

url_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
url_names = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names'

response_data = requests.get(url_data)
response_names = requests.get(url_names)

# Guardar los archivos descargados
with open('adult.data', 'wb') as f:
    f.write(response_data.content)

with open('adult.names', 'wb') as f:
    f.write(response_names.content)

# Leemos datos
with open(os.path.join(os.getcwd(),'adult.data'),'r') as f:
    data = f.read().splitlines() # Dividimos el texto por saltos de línea
    data = [elem.split(',') for elem in data] # Dividimos cada línea por las comas y removemos líneas vacías

# Leemos metadata
with open(os.path.join(os.getcwd(),'adult.names'),'r') as f:
    metadata = f.read().splitlines()

# Regex
## Buscamos palabras que empiecen por letras mayús. o minús. de duración variable y que tengan dos puntos
regex_fn = lambda text: re.findall('^[a-zA-Z-]+:{1}', text)  
## Buscamos palabras con letras mayús. o minús. de duración variable
reg_text_fn = lambda text : re.findall('[a-zA-Z- ]+', text)  

# Aplicamos la expresión regular en forma de lambda a al metadata
# Téngase en cuenta que el método findall devuelve una lista vacía si ninguna expresión coincide con el patrón introducido

metadata_list = [regex_fn(elem)[0] for elem in metadata if regex_fn(elem)]
col_names = [reg_text_fn(elem)[0] for elem in metadata_list if reg_text_fn(elem)] + ['label']

# Construimos el objeto pd.DataFrame
df_ADULT = pd.DataFrame(data=data, columns=col_names)
df_ADULT

Comenzamos el análisis de los datos buscando su información general

In [ ]:
df = df_ADULT.copy() # Copiamos el dataframe para no modificar el original y trabajar con una variable de nombre corto.
df.info() # Como se puede apreciar, todas las columnas son de "Dtype: object", lo que quiere decir que probablemente sean strings, cosa que no tiene
# sentido, por ejemplo, para la edad. De hecho, si vamos a la fuente, observaremos lo siguiente:

Descripción de las variables:
- age: continuous.
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt: continuous.
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num: continuous.
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex: Female, Male.
- capital-gain: continuous.
- capital-loss: continuous.
- hours-per-week: continuous.
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

Por lo pronto, vamos a ver el aspecto de los primeros datos

In [ ]:
df.head() # Al menos 6 de las columnas deben ser numéricas tal y como está indicado en adult.names (continuous)
# Estas variables son: age, fnlwgt, education-num, capital-gain, capital-loss y hours-per-week.

Recordatorio: una forma de hacerlo directamente con **todas** las variables que se pueda es la siguiente:
```python
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')

df.info()
```

In [ ]:
# Transformación del tipo de las variables a numérico:

df["age"] = df["age"].astype(int)
df["fnlwgt"] = df["fnlwgt"].astype(int)
df["education-num"] = df["education-num"].astype(int)
df["capital-gain"] = df["capital-gain"].astype(int)
df["capital-loss"] = df["capital-loss"].astype(int)
df["hours-per-week"] = df["hours-per-week"].astype(int)

# Finalmente, comprobamos que las variables son números ahora:
df.info()

# Obtenemos un error porque uno de los valores, además de ser un string, es un string vacío, por lo que no puede ser transformado a int.

Vamos entonces a estudiar los valores nulos de las variables

In [ ]:
df["age"].unique() # Se puede ver que uno de los valores es un vacío, por lo que no vamos a poder convertir los datos de string a número.

In [ ]:
df["age"].isnull().value_counts() # Vemos que no hay valores nulos, sino un string vacío, asi que vamos a corregir esto

In [ ]:
df["age"] #Parece que es el último dato el que falla, vamos a ver las últimas filas 

In [ ]:
df.tail() # En efecto, en la base de datos hay una fila extra vacía que provoca fallos y no aporta información, así que vamos a borrarla.

In [ ]:
#Para borrar la última fila, podemos hacer lo siguiente:
df.index[-1] # Nos da el índice de la fila del final, comprobamos que es la 32561
df = df.drop(df.index[32561]) # Borramos la fila del final, la 32561
df.tail()

# Este código solo va a funcionar una vez pues la fila 32561 ya no existe, pero si indicásemos la fila del final con un -1, se borraría la última fila
# constantemente al ejecutar este comando.

In [ ]:
# Una forma de que no pase lo anterior sería la siguiente:
df = df_ADULT.copy() # Explicitamos esta línea aquí para generar el df de nuevo.
df = df.drop(df.index[-1])  # Borramos la última fila.
df.tail() # Ahora siempre se verá que la última fila es la número 32560 da igual cuantas veces se ejecute el código.

In [ ]:
# Transformación del tipo de las variables a numérico:

df["age"] = df["age"].astype(int)
df["fnlwgt"] = df["fnlwgt"].astype(int)
df["education-num"] = df["education-num"].astype(int)
df["capital-gain"] = df["capital-gain"].astype(int)
df["capital-loss"] = df["capital-loss"].astype(int)
df["hours-per-week"] = df["hours-per-week"].astype(int)

# Finalmente, comprobamos que las variables son números ahora:
df.info()

Hasta aquí, hemos eliminado todos los valores nulos, ahora hay que comprobar si los valores no-nulos son valores válidos o no.

In [ ]:
# De forma un poco rudimentaria, pero útil a modo de práctica, podemos emplear el comando .unique() para ver los valores únicos de cada columna

#print(df["age"].unique()) # Todo correcto, ponemos un # para que la línea no se ejecute 

#####print(df["workclass"].unique()) # Problemas, hay una categoría de datos que es "?"

#print(df["fnlwgt"].unique()) # No se pueden ver todos los datos, hace falta otro método, como un groupby. ### Comprobado que está bien.

#print(df["education"].unique()) # Todo correcto

#print(df["education-num"].unique()) # Todo correcto

#print(df["marital-status"].unique()) # Todo correcto

#####print(df["occupation"].unique()) # Hay una categoría "?"

#print(df["relationship"].unique()) # Todo correcto

#print(df["race"].unique()) # Todo correcto

#print(df["sex"].unique()) # Todo correcto

#print(df["capital-gain"].unique()) # Todo correcto

#print(df["capital-loss"].unique()) # Todo correcto

#print(df["hours-per-week"].unique()) # Todo correcto, aunque hay gente que trabaja más de 90h a la semana, pobrecitos

#####print(df["native-country"].unique()) # Hay una categoría "?"

#print(df["label"].unique()) # Todo correcto

In [ ]:
# Una forma más rápida y directa es con el siguiente comando:
for col in df.columns: # Para cada columna del dataframe:
    print(df[col].unique()) # Imprimimos los valores/categorías únicas de cada columna
    print("\n") # y un espacio de línea

Una forma rápida de agrupar las variables es con el código que viene a continuación:  

```python
# Sacamos los nombres de las columnas numéricas y categóricas
def tipo_de_columnas_ordenadas (df):
    cat = []
    num = []
        
    for col in df.columns:
        if(df[col].dtype == "object"):
            cat.append(col)
        else:
            num.append(col)

    return cat , num

cat , num = tipo_de_columnas_ordenadas(df)
print("Las columnas categóricas son: ", cat)
print("Las columnas numéricas son: ", num)

# Es una buena forma de agrupar las variables según su tipo, especialmente cuando tenemos un número elevado de columnas, aunque requiere de una buena
# limpieza previa de los datos.
```

Vamos a estudiar los casos específicos de "workclass", "fnlwgt", "occupation" y "native-country"

In [ ]:
df["workclass"].value_counts() # Hay 1836 valores que no tienen una "?"

In [ ]:
print(df.sort_values(by = "fnlwgt")) # En la columna fnlwgt no hay valores extraños como "?" ni cerca del mínimo ni del máximo, por lo que esta variable está bien

In [ ]:
df["occupation"].value_counts() # Hay 1843 valores que no tienen valores de profesión

In [ ]:
df["native-country"].value_counts() # Hay 583 personas que no tienen país de origen

Vamos a eliminar entonces los datos indeterminados y sustituirlos por valores nulos

In [ ]:
# Reemplazamos ' ?' (si, con espacio) por nulo
df = df.replace(' ?',np.NaN)

# Vemos qué columnas tienen datos faltantes
missing_cols = list(df.isnull().sum(axis=0)[df.isnull().sum(axis=0)>0].index)

# Filtramos las filas donde hay algún dato nulo, y las columnas donde están
df.loc[df.isnull().sum(axis=1)>0, missing_cols]

In [ ]:
df.info()

Ahora tenemos valores nulos en Workclass, Occupation y Native-Country. ¿Qué podemos hacer con ellos? Al ser variables atributos (no numéricas, sino palabras, categóricas) la mejor opción es asignarles la moda a esos valores, que son "Private", "Prof-Specialty" y "USA" respectivamente.

```python
def replace_missing_data(df):
    # Vemos qué columnas tienen valores nulos
    mis_cols = list(df.isnull().sum(axis=0)[df.isnull().sum(axis=0)>0].index)
    # Iteramos sobre ellas
    for col in mis_cols:
        # Si la variable es discreta,...
        if df[col].dtype in ['object']:
            mode_col = df[col].mode().values[0]
            df[col] = df[col].fillna(mode_col)
        # Si son números enteros
        elif df[col].dtype in ['int']:
            df[col] = df[col].fillna(df['col'].median())
        # Si son números reales
        elif df[col].dtype in ['float']:
            df[col] = df[col].fillna(df['col'].mean())
    # Devolvemos el DataFrame
    return df
```

In [ ]:
def replace_missing_data(df):
    # Vemos qué columnas tienen valores nulos
    mis_cols = list(df.isnull().sum(axis=0)[df.isnull().sum(axis=0)>0].index)
    # Iteramos sobre ellas
    for col in mis_cols:
        # Si la variable es discreta,...
        if df[col].dtype in ['object']:
            mode_col = df[col].mode().values[0]
            df[col] = df[col].fillna(mode_col)
        # Si son números enteros
        elif df[col].dtype in ['int']:
            df[col] = df[col].fillna(df['col'].median())
        # Si son números reales
        elif df[col].dtype in ['float']:
            df[col] = df[col].fillna(df['col'].mean())
    # Devolvemos el DataFrame
    return df

replace_missing_data(df)
print(df.info()) # Hemos solucionado todos los errores nulos
df[["workclass", "occupation", "native-country"]][60:63] # Antes había problemas en workclass, occupation y native-country en la linea 61

Ya hemos corregido nuestra base de datos. Hemos eliminado celdas vacías y hemos sustituido los valores nulos por la moda de sus respectivas columnas al tratarse de variables atributo (palabras). Nuestra base de datos ya está lista para ser estudiada y analizada por los expertos 😎👌

Recordatorio: Código para discretizar variables:

```python
  # Tomamos el mínimo y máximo de los datos
  min_col, max_col = df['engine-size'].min(), df['engine-size'].max()
  # Decidimos en cuántas cajas vamos a estratificar los datos
  num_boxes = 8
  # Creamos los valores que segmentarán las cajas
  bins = np.linspace(min_col, max_col, num_boxes+1)
  # Creamos la columna discretizada
  df['engine-size-disc'] = np.digitize(df['engine-size'], bins)
```

### Análisis estadístico de los datos (Explayarse todo lo que uno quiera)

In [ ]:
# Intervalo de edad
intervalo_edad = df["age"].max()-df["age"].min()

#Histograma con la edad
df["age"].hist(bins = intervalo_edad, color = "green", edgecolor = "grey")